# VacationPy
----

#### Note
* This notenook utilizes Google API. Keep an eye on API utilization. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.


In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Getting Weather Data from WeatherPy results into DataFrame

In [24]:
# Getting weather data from WeatherPy work and storing in a dataframe
weather_data_file = "../output_data/cities.csv"
weather_data = pd.read_csv(weather_data_file)
# Displaying preview of weather data
weather_data.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,Nikolskoye,59.70,30.79,37.40,93,90,8.95,RU,1603308261
1,1,Waipawa,-41.41,175.52,57.00,76,8,11.41,NZ,1603308530
2,2,East London,-33.02,27.91,66.20,77,2,11.41,ZA,1603308530
3,3,Kuching,1.55,110.33,75.20,94,40,2.24,MY,1603308531
4,4,Rikitea,-23.12,-134.97,73.87,66,5,8.52,PF,1603308531


### Humidity Heatmap

In [25]:
# Configuring gmaps with API key
gmaps.configure(api_key=g_key)

# Defining parameters to be passed to gmaps api
locations = weather_data[["Lat", "Lon"]].astype(float)
humidity = weather_data["Humidity"].astype(float)
max_humidity = weather_data["Humidity"].max()

In [26]:
# Configuring gmaps layout
fig = gmaps.figure(layout={
        'width': '960px',
        'height': '600px',
        'padding': '3px'
    })

# Creating a Humidity Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True, max_intensity=max_humidity)

# Adding Heatmap layer to the map
fig.add_layer(heat_layer)

# Displaying the map
fig

Figure(layout=FigureLayout(height='600px', padding='3px', width='960px'))

### Selection of Vacation Cities
#### Criteria for selection of vacation ciity
* A max temp < 80 degree and > 70 degree
* Wind speed < 10 mph
* Zero cloudiness


In [27]:
# Storing vacation city data in a dataframe with above listed criteria
vacation_data = weather_data[(weather_data["Max Temp"]<80.00)
                            & (weather_data["Max Temp"]>70.00)
                            & (weather_data["Windspeed"]<10.00)
                            & (weather_data["Cloudiness"]==0)]

# Removing NA values from the dataframe
vacation_data = vacation_data.dropna()

# Displaying the final list of Vacation City list
vacation_data

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
197,197,Yaan,7.38,8.57,78.46,78,0,2.95,NG,1603308566
208,208,Thabazimbi,-24.59,27.41,73.99,48,0,1.01,ZA,1603308567
252,252,Monte Patria,-30.69,-70.95,79.34,25,0,8.79,CL,1603308575
291,291,Alice Springs,-23.70,133.88,78.80,39,0,8.05,AU,1603308520
318,318,Dum Duma,27.57,95.57,71.10,96,0,2.35,IN,1603308586
319,319,Kirkuk,35.47,44.39,73.09,21,0,8.93,IQ,1603308587
321,321,Dharān,26.81,87.28,71.58,76,0,5.46,NP,1603308587
367,367,Upington,-28.45,21.26,71.60,13,0,4.70,ZA,1603308595
370,370,Makkah al Mukarramah,21.43,39.83,78.08,36,0,4.34,SA,1603308595
371,371,Ýpsonas,34.69,32.96,75.20,57,0,8.05,CY,1603308596


### Hotel Map

In [28]:
# Finding the closest hotel of each type to coordinates

# Setting up baseurl for api call and radius for searching Hotel for Vacation City
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000

# Running a loop through all cities in Vacation City list to find Hotels
for index, row in vacation_data.iterrows():
    lat = row["Lat"]
    lon = row["Lon"]
    city = row["City"]
   
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "type": "lodging",
        "key": g_key
    }
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    
    # Handling exceptions in case no Hotel found for a City and adding Hotel Name in Vacation Dataframe
    try:
        # Getting first hotel name from the list of hotels received from api call
        hotel = results[0]["name"]
        print(f"Closest Hotel to city {city} is {hotel}.")
        vacation_data.loc[index, "Hotel Name"] = hotel
        
    except (KeyError, IndexError):
        print("Could not find a nearby hotel...skipping")

Could not find a nearby hotel...skipping
Closest Hotel to city Thabazimbi is eagels rest.
Closest Hotel to city Monte Patria is Hotel Monte Cordillera.
Closest Hotel to city Alice Springs is Desert Palms Alice Springs.
Closest Hotel to city Dum Duma is Komal bakery shope.
Closest Hotel to city Kirkuk is فندق التأميم.
Closest Hotel to city Dharān is Nirvana Coutry Club Health & Golf Resorts.
Closest Hotel to city Upington is River Place Manor.
Closest Hotel to city Makkah al Mukarramah is Hotel Fairmont Makkah Clock Royal Tower.
Closest Hotel to city Ýpsonas is Club Aphrodite.
Closest Hotel to city Kiryat Gat is OlusHome.
Closest Hotel to city Vallenar is Humacao Bed & Breakfast.
Closest Hotel to city Najrān is محمد هادي ال فطيح.
Closest Hotel to city Maragogi is Salinas do Maragogi All Inclusive Resort.


In [29]:
# Removing cities from Vacation Cities, where no hotel has been found
hotel_df = vacation_data.dropna()

# Displaying final list of Vacation city with Hotel Name
hotel_df

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
208,208,Thabazimbi,-24.59,27.41,73.99,48,0,1.01,ZA,1603308567,eagels rest
252,252,Monte Patria,-30.69,-70.95,79.34,25,0,8.79,CL,1603308575,Hotel Monte Cordillera
291,291,Alice Springs,-23.70,133.88,78.80,39,0,8.05,AU,1603308520,Desert Palms Alice Springs
318,318,Dum Duma,27.57,95.57,71.10,96,0,2.35,IN,1603308586,Komal bakery shope
319,319,Kirkuk,35.47,44.39,73.09,21,0,8.93,IQ,1603308587,فندق التأميم
321,321,Dharān,26.81,87.28,71.58,76,0,5.46,NP,1603308587,Nirvana Coutry Club Health & Golf Resorts
367,367,Upington,-28.45,21.26,71.60,13,0,4.70,ZA,1603308595,River Place Manor
370,370,Makkah al Mukarramah,21.43,39.83,78.08,36,0,4.34,SA,1603308595,Hotel Fairmont Makkah Clock Royal Tower
371,371,Ýpsonas,34.69,32.96,75.20,57,0,8.05,CY,1603308596,Club Aphrodite
374,374,Kiryat Gat,31.61,34.76,77.00,88,0,7.92,IL,1603308596,OlusHome


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [31]:
# Configuring marker layer for display with Heatmap layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Adding marker layer to existing heatmap figure
fig.add_layer(marker_layer)

# Displaying updated figure
fig


Figure(layout=FigureLayout(height='600px', padding='3px', width='960px'))